# Crawling

In [1]:
## 크롤링 패키지
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm, trange

## 전처리 패키지
import pandas as pd
import re

## 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 에러 메시지 
import warnings
warnings.filterwarnings("ignore")

## 1. NAVER News(8/1 ~ 8/31 : 정치, 경제, 사회, 생활 문화)

### <span style = "background : yellow">- TOP 30 HeadLine

- 31 days x 30 news x 4 categories = 3,720 rows

In [2]:
browser = Chrome()

In [3]:
title_list = [] # 기사 제목
text_list = [] # 기사 본문
date_list = [] # 기사 날짜

In [4]:
for id in [100, 101, 102, 103] : # 뉴스 카테고리 지정
    for date in range(801, 832) : # 뉴스 기사 날짜 지정
        # url 설정
        url = "https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId={}".format(id)
        url = url + "&date=20200{}".format(date)
        browser.get(url)
        time.sleep(.5)
        for num in range(1, 31) : # 기사가 30개이므로 1 ~ 31 지정
            date_list.append(date) # 뉴스 기사 날짜 추가
            # 해당 뉴스 기사 클릭
            browser.find_element_by_css_selector("#wrap > table > tbody > tr > td.content > div > div.ranking > ol > li.ranking_item.is_num{} > div.ranking_text > div.ranking_headline > a".format(num)).click()
            time.sleep(.75)
            # 기사 제목 및 본문 추가
            try:
                title_list.append(browser.find_element_by_css_selector("#articleTitle").text)
                text_list.append(browser.find_element_by_css_selector("#articleBodyContents").text)
            except :
                try :
                    title_list.append(browser.find_element_by_css_selector("#articleTitle").text)
                    text_list.append(browser.find_element_by_css_selector("#newsEndContents").text)
                except: 
                    try :
                        title_list.append(browser.find_element_by_css_selector("#content > div.end_ct > div > h2").text)
                        text_list.append(browser.find_element_by_css_selector("#articeBody").text)
                    except :
                        title_list.append("재크롤링요망")
                        text_list.append('재크롤링요망')
            browser.get(url)
            time.sleep(.75)

In [5]:
len(title_list), len(date_list), len(text_list)

(3720, 3720, 3720)

In [6]:
df = pd.DataFrame({'제목' : title_list, '내용' : text_list, '일자' : date_list})

In [9]:
df.to_csv("naver_news_august.csv", index = False, encoding = 'utf-8')

In [11]:
df.sample(2)

,제목,내용,일자
2360,2박 3일 여행 후 확진 판정…제주 여행 확진자 동선 공개,[KBS 제주]\n다른 지역 코로나19 확진자가 최근 제주 여행을 다녀온 것이 확인...,817
3377,[속보] 사랑제일교회 간 학부모 통해 음악연습실 2명 감염,사택 나서는 전광훈 목사 [사진=연합뉴스]\n\n전광훈 목사가 대표로 있는 사랑제일...,820


## 2. 비대면 문화 향유

<span style = "background : pink">(COVID19에 따른 비대면 문화향유 현황 2월 20일~)
    
#### 여행/레저, 음식/맛집, 공연/전시 카테고리 기사 크롤링

In [2]:
from datetime import datetime, timedelta

In [3]:
now = datetime.now()

In [4]:
dates = sorted([(now + timedelta(days = -days)).strftime('%m%d') for days in range(1, 208)])

In [78]:
browser = Chrome()

In [79]:
content_list = []
date_list = []

In [80]:
for date in dates :
    url = 'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=242&sid1=103&mid=shm&date=2020{}'.format(date)
    print("날짜 {} 기사 크롤링 시작!".format(date))
    print("#" * 60)
    browser.get(url)
    time.sleep(.4)
    try:
        pages = int(browser.find_element_by_css_selector("#main_content > div.paging").text[-1])
    except :
        browser.find_element_by_css_selector("#main_content > div.paging > a.next.nclicks\(fls\.page\)").click()
        pages = pages = int(browser.find_element_by_css_selector("#main_content > div.paging").text.split(' ')[-1])
    for page in range(1, pages + 1):
        browser.get(url + "&page={}".format(page))
        print('{}번째 페이지 시작'.format(page))
        time.sleep(.5)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        for i in range(1, 11):
            try:
                content_url = soup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li:nth-child({}) > dl > dt:nth-child(2) > a'.format(i))[0]['href']
                browser.get(content_url)
                time.sleep(.7)
                content = browser.find_element_by_css_selector("#articleBodyContents").text
                content_list.append(content)
                print(i, ' / 내용 = {}'.format(content[:20]))
                browser.get(url+ "&page={}".format(page))
                time.sleep(.5)
                date_list.append(date)
            except :
                try :
                    content_url = soup.select("#main_content > div.list_body.newsflash_body > ul.type06_headline > li:nth-child({}) > dl > dt > a".format(i))[0]['href']
                    browser.get(content_url)
                    time.sleep(.7)
                    content = browser.find_element_by_css_selector("#articleBodyContents").text
                    content_list.append(content)
                    print(i, ' / 내용 = {}'.format(content[:20]))
                    browser.get(url+ "&page={}".format(page))
                    time.sleep(.5)
                    date_list.append(date)
                except :
                    pass
        for i in range(1, 11):
            try:
                content_url = soup.select('#main_content > div.list_body.newsflash_body > ul.type06 > li:nth-child({}) > dl > dt:nth-child(2) > a'.format(i))[0]['href']
                browser.get(content_url)
                time.sleep(.7)
                content = browser.find_element_by_css_selector("#articleBodyContents").text
                content_list.append(content)
                print(i + 10, ' / 내용 = {}'.format(content[:20]))
                browser.get(url+ "&page={}".format(page))
                time.sleep(.5)
                date_list.append(date)
            except :
                try:
                    content_url = soup.select("#main_content > div.list_body.newsflash_body > ul.type06 > li:nth-child({}) > dl > dt > a".format(i))[0]['href']
                    browser.get(content_url)
                    time.sleep(.7)
                    content = browser.find_element_by_css_selector("#articleBodyContents").text
                    content_list.append(content)
                    print(i+10, ' / 내용 = {}'.format(content[:20]))
                    browser.get(url+ "&page={}".format(page))
                    time.sleep(.5)
                    date_list.append(date)
                except :
                    pass
        print('{}번째 페이지 완료'.format(page))
        print('*' * 50)
    print("날짜 {} 기사 크롤링 끝!".format(date))

날짜 0220 기사 크롤링 시작!
############################################################
1번째 페이지 시작
1  / 내용 = 재개관일은 미정
국립대구박물관 코로나
2  / 내용 = 6월28일 롯데콘서트홀에서


[아시
3  / 내용 = 이데일리 창립 20주년 일자리 창출 
4  / 내용 = 열감지카메라로 발열 체크
손소독제·마
5  / 내용 = [이데일리 방인권 기자] 가수 양희은
6  / 내용 = [이데일리 방인권 기자] 가수 양희은
7  / 내용 = [이데일리 방인권 기자] 가수 양희은
8  / 내용 = [이데일리 방인권 기자] 가수 더원과
9  / 내용 = [이데일리 방인권 기자] 가수 남주희
10  / 내용 = [이데일리 방인권 기자] 가수 더원과
11  / 내용 = 국내 가수들도 해외 투어 연기·취소

12  / 내용 = ㆍ내일 취임 후 첫 국내 리사이틀…드
13  / 내용 = [이데일리 방인권 기자] 가수 더원이
14  / 내용 = [이데일리 방인권 기자] 가수 더원이
15  / 내용 = [이데일리 방인권 기자] 가수 더원이
16  / 내용 = [이데일리 방인권 기자] 소프라노 신
17  / 내용 = [이데일리 방인권 기자] 소프라노 신
18  / 내용 = [이데일리 방인권 기자] 기타리스트 
19  / 내용 = [이데일리 방인권 기자] 기타리스트 
20  / 내용 = [이데일리 방인권 기자] 기타리스트 
1번째 페이지 완료
**************************************************
2번째 페이지 시작
1  / 내용 = [이데일리 방인권 기자] 20일 오후
2  / 내용 = 화랑미술제 부스도 '빈 채' 불참
'
3  / 내용 = 10월에는 남정호 신임 예술감독 신작
4  / 내용 = 코로나19 확산 방지를 위해 방역 작
5  / 내용 = 문화체육관광부가 코로나19 사태에 다
6  / 내용 = [합천=뉴시스] 김기진 기자 = 경남
7  / 내용 = [아시아경제 이종

1  / 내용 = 연극 ‘아버지와 나와 홍매와’ 신구·
2  / 내용 = 영화관객 3분의1로 감소, 2013년
3  / 내용 = 오는 28일부터 아트선재센터에서



4  / 내용 = [라이브네이션 제공]


[헤럴드경제
5  / 내용 = (화천=연합뉴스) 이상학 기자 = 강
6  / 내용 = 세계적 작가 레안드로 에를리치展

북
7  / 내용 = 美 레코딩아카데미 인터뷰
"`기생충`
8  / 내용 = [헤럴드경제=고승희 기자] 신종 코로
9  / 내용 = [정규 4집 ‘맵 오브 더 솔: 7’
10  / 내용 = [개인전 ‘백 비트!’ 연 권혜성 작
11  / 내용 = 문화제조창C
[청주=뉴시스] 강신욱 
12  / 내용 = 삼국지 영웅들의 이야기를 칼군무와 힘
13  / 내용 = 예매건수 급감...폐막일 앞두고 취소
14  / 내용 = 대구시립교향악단 단원들과 줄리안 코바
15  / 내용 = 3월 10일 ~ 3월 15일, 대구 
16  / 내용 = 연극 '우리 노래방 가서 얘기 좀 할
18  / 내용 = 국악계 괴짜 슈퍼스타 이희문

소리꾼
19  / 내용 = [강릉=뉴시스]김경목 기자 = 22일
20  / 내용 = [CBS노컷뉴스 배덕훈 기자]
(사진
1번째 페이지 완료
**************************************************
2번째 페이지 시작
1  / 내용 = 25일까지 계획된 공연 모두 취소


2  / 내용 = '라디오프랑스필하모닉' 첫 동양인 악
3  / 내용 = (서울=연합뉴스) 송광호 기자 = 신
4  / 내용 = (서울=연합뉴스) 류효림 기자 = 2
5  / 내용 = (서울=연합뉴스) 류효림 기자 = 2
6  / 내용 = (서울=연합뉴스) 류효림 기자 = 2
7  / 내용 = (서울=연합뉴스) 류효림 기자 = 2
8  / 내용 = (서울=연합뉴스) 류효림 기자 = 2
9  / 내용 = 예매건수 급감...폐막일 앞두고 취소
10  / 내용 = ‘나는 너를 만들고, 너는 나를 그린
11  / 내용 = (수원=연합

12  / 내용 = 국립극단 이성열 ‘화전가’- 극단물리
13  / 내용 = 신시컴퍼니, 공연 일정 변경·취소 결
14  / 내용 = 김성환 화백 유족, 유품 5천여점 군
15  / 내용 = 김민조, helper, oil on 
16  / 내용 = (서울=연합뉴스) 이승우 기자 = 도
17  / 내용 = 연극 ‘아버지와 나와 홍매와’의 한 
18  / 내용 = (재)원주문화재단은 '달마다 루프탑페
19  / 내용 = 만 40세 이하 청년, 신진작가 대상
20  / 내용 = ‘빌보드 200’ 1위 전망…네번째 
4번째 페이지 완료
**************************************************
5번째 페이지 시작
1  / 내용 = (군포=연합뉴스) 지난해 9월 8일 
2  / 내용 = 루체른스트링페스티벌 내한공연 취소 공
3  / 내용 = 협회장 "축제 예산 21억원, 협회 
4  / 내용 = 3월 초순까지 클래식·무용 등 공연 
5  / 내용 = JTBC '슈퍼밴드'에서 독보적인 비
6  / 내용 = 그룹 드림캐쳐가 의문의 마스크 댄서로
8  / 내용 = 영국 오피셜 차트도 1위 예약

방탄
9  / 내용 = 솔로 앨범 `랑데부 인 파리` 발매 
10  / 내용 = (20) 그림이 ‘증거’가 될 수 있
11  / 내용 = (옥천=연합뉴스) 심규석 기자 = 올
12  / 내용 = 극단 학전 영상노래극
김민기 작사·작
13  / 내용 = [아시아경제 박병희 기자] 금호아트홀
14  / 내용 = [CBS노컷뉴스 곽인숙 기자]
국립극
15  / 내용 = 7개 국립예술단체 공연도 잠정 중단

16  / 내용 = 생상스·포레·드뷔시·풀랑 등 프랑스 
17  / 내용 = [서울경제] 금호아트홀 연세는 오는 
18  / 내용 = 블루스퀘어 ‘스테이지B’ 해피베리아워
19  / 내용 = "코로나 19 확산 예방에 공연예술계
20  / 내용 = [헤럴드경제=고승희 기자] 문화체육관
5번째 페이지 완료
****************************

9  / 내용 = 당진 기지시줄다리기 민속축제 [연합뉴
10  / 내용 = 음악학자 로버트 가피아스 연구집·구술
11  / 내용 = 내달 6~8일 '판소리 햄릿-송보라 
12  / 내용 = 더 팩트 뮤직 어워즈 (사진=2020
13  / 내용 = 첼로곡 담은 '랑데부 인 파리' 발매
14  / 내용 = - [엄홍현 EMK뮤지컬컴퍼니 대표]
15  / 내용 = 의혹 반년간 부인하다 “미안” … 음
16  / 내용 = 신창호 한예종 무용원 교수
4월 신작
17  / 내용 = 세계 3대 카니발 중 하나…가장 화려
18  / 내용 = 스페인 국립공연예술연구소, 마드리드 
19  / 내용 = 새를 그려넣은 `春川' ◇원혁로 作 
20  / 내용 = - 공연 취소 결정 어려운 민간 예술
5번째 페이지 완료
**************************************************
날짜 0227 기사 크롤링 끝!
날짜 0228 기사 크롤링 시작!
############################################################
1번째 페이지 시작
1  / 내용 = KBS 2TV ‘뮤직뱅크’ 제공

K
2  / 내용 = [아시아경제 박병희 기자] 메가박스가
3  / 내용 = 국악방송 '고전의 숨결-전태용을 만나
4  / 내용 = 2월 22일~2월 28일

◇전시


5  / 내용 = 부이사장 한호(왼쪽부터), 이사장 김
6  / 내용 = 플라시도 도밍고. 스포츠서울DB
[스
7  / 내용 = [세기말의 '팜므 파탈']
남녀평등·
8  / 내용 = 조시아나 여공작 역 신영숙. 제공|E
9  / 내용 = [서울=뉴시스] 그린데이. (사진 =
10  / 내용 = 태권도·양궁처럼 육성시스템 수출
각국
11  / 내용 = 코로나 여파 4월 4차례 공연 취소

14  / 내용 = (서울=연합뉴스) 이재희 기자 = 2
15  / 내용 = (서울=연합뉴스) 이재희 기자 = 2
16  / 내용 = (서울=연합뉴스) 이재희 기자 = 2
17  / 내

15  / 내용 = 설휘 갤러리조은·박신영 교보문고 합정
16  / 내용 = (서울=연합뉴스) 송광호 기자 = 국
17  / 내용 = [2년 만에 새 앨범 낸 새소년]
‘
18  / 내용 = 뮤지컬 마리 퀴리 (사진=쇼온컴퍼니)
19  / 내용 = [서울=뉴시스] '오늘, 식민지로 살
20  / 내용 = [헤럴드경제=고승희 기자] 코로나바이
1번째 페이지 완료
**************************************************
2번째 페이지 시작
1  / 내용 = (서울=연합뉴스) 송광호 기자 = 예
2  / 내용 = 【서울=뉴시스】 예술의전당 오페라하우
3  / 내용 = 자체 기획공연·전시·강의 '올 스톱'
4  / 내용 = [헤럴드경제=고승희 기자] 3·1절 
5  / 내용 = [CBS노컷뉴스 곽인숙 기자]
국립발
6  / 내용 = 지난해 울산 무거천 벚꽃[연합뉴스 자
7  / 내용 = 양주 회암사지 왕실축제 '어가행렬'[
8  / 내용 = 국립발레단이 자가격리 기간 중 일본 
9  / 내용 = [서울=뉴시스]이재훈 기자 = 신종 
10  / 내용 = [위기 맞은 문화예술계 긴급진단]
유
11  / 내용 = [서울=뉴시스]교보문고 합정점 아트월
12  / 내용 = [서울=뉴시스] 내부감사품질 국제공인
13  / 내용 = 김준수가 쓰레기 줍기 챌린지에 참여했
14  / 내용 = [헤럴드경제=고승희 기자] 공연예술 
15  / 내용 = '마리 퀴리' 실황 녹화 중계 21만
16  / 내용 = 대구거주 여성 22일 `셜록홈즈` 관
17  / 내용 = [라이브 제공]


[헤럴드경제=고승
18  / 내용 = 발레리노 나대한. 국립발레단 제공


19  / 내용 = 셰익스피어 원작 재구성
'조씨고아' 
20  / 내용 = 4∼29일 대학로 소극장 혜화당

'
2번째 페이지 완료
**************************************************
3번째 페이지 시작
1  / 내용 = [서울=뉴시스] 연극 '리어외전' 포


...

4  / 내용 = 시, 코로나19 진정 시 소규모 문화
5  / 내용 = [서울=뉴시스] 테너 김민석. 202
6  / 내용 = [서울=뉴시스] 문화예술상 시상식에서
7  / 내용 = '7080' 향수 소환한 광주 동구 
8  / 내용 = 장애인 특수학교 둘러싼 논쟁 소재
현
9  / 내용 = 완주 소싸움 대회
[연합뉴스 자료사진
10  / 내용 = 안진석 작


코로나19로 인한 생활
11  / 내용 = 제6회 DIMF 뮤지컬스타 경연 모습
12  / 내용 = `부릉부릉 수묵시동’ 내년 비엔날레 
13  / 내용 = 국립국악원이 1일 개설한 국악기 디지
14  / 내용 = 뮤지컬 '개와 고양이의 시간' 1일~
15  / 내용 = 2020부산비엔날레 온라인 홍보물 스
16  / 내용 = [서울=뉴시스]가상현실 온라인 전시 
17  / 내용 = [서울=뉴시스] 국립극장 전경. (사
18  / 내용 = 지용호 개인전, 가나아트센터 13일까
19  / 내용 = [부산=뉴시스] 허상천 기자 = 부산
3번째 페이지 완료
**************************************************
4번째 페이지 시작
1  / 내용 = 강서구 ‘장애인학교 설립’ 토론회 바
2  / 내용 = 취소수수료 없이 금액 전액환불
전화·
3  / 내용 = 박불똥, 5-정물, 디아섹에 C-Pr
5  / 내용 = (서울=연합뉴스) 국립국악원이 국악기
6  / 내용 = 김시영 작가의 2020년작 <알마즈>
7  / 내용 = 2일부터 '2020 헬로, 케이!' 
8  / 내용 = '제6회 동편제마을 국악 거리축제'

9  / 내용 = (서울=연합뉴스) 국내 근현대 시를 
10  / 내용 = 문화유산 아이디어 공모전 최우수상에 
11  / 내용 = [CBS노컷뉴스 권혁주 기자]
(사진
12  / 내용 = 광화문 교보아트스페이스, 김소월 등단
14  / 내용 = (서울=연합뉴스) 김보경 기자 = 현
16  / 내용 = 조옥현 도의원 대표발의
조옥현 전남도
17  / 내용 = 현대

5번째 페이지 시작
1  / 내용 = 클라라 주미 강, 손열음 [크레디아 
2  / 내용 = 고창동리국악교실서 처음 판소리 배워…
3  / 내용 = 창작가무극 '잃어버린 얼굴 1895'
4  / 내용 = 20여년 탐구 골드베르크 변주곡, 깊
5  / 내용 = (수원=연합뉴스) 김인유 기자 = 경
6  / 내용 = 뮤지컬 ‘더픽션’. 제공|HJ컬쳐
[
7  / 내용 = 中 제작자와 라이선스 계약
현실에 나
8  / 내용 = '바흐: 골드베르크 변주곡' 앨범 발
9  / 내용 = 출범 공연으로서 의미 커…최상 컨디션
10  / 내용 = 오는 12월 상하이서 라이선스 공연

11  / 내용 = ◇뮤지컬 '더 픽션' 공연 장면. 사
12  / 내용 = [아이뉴스24 박은희 기자] 뮤지컬 
13  / 내용 = 뮤지컬 '더픽션 [HJ컬쳐 제공]


14  / 내용 = 코로나19로 불확실성 커져 '순연' 
15  / 내용 = HJ컬쳐, 中 상하이 씨뮤지컬과 계약
16  / 내용 = 5톤 트럭 개조한 이동공연장 ‘더윙’
17  / 내용 = 8월 공연 매출, 전년比 41% '급
18  / 내용 = 안내원 1m 거리 유지 빈틈없이 안내
19  / 내용 = [요요 미술기행-50] 김영궁은 대학
20  / 내용 = '코로나19 서울 공공미술 프로젝트'
5번째 페이지 완료
**************************************************
6번째 페이지 시작
1  / 내용 = ‘모차르트!’ 추석연휴 관람권 판매…
2  / 내용 = (18) 김승영 ‘누구나 마음속에 정
3  / 내용 = 2020년 작
조선 이덕무 만든 밀랍
4  / 내용 = [서울=뉴시스] 오동현 기자 = 넥슨
5  / 내용 = 아버지를 죽인 어머니를 처단하려는 딸
6  / 내용 = 옥스퍼드大 박물관 전시 예정
17세기
7  / 내용 = 시백 안종중 화백 개인전이 2일 강원
8  / 내용 = 손바닥 위 작은 미술·전시관 ◇백선옥
9  / 내용 = 춘천MBC '한국현대조각초대전' ◇김
10  / 

2번째 페이지 완료
**************************************************
3번째 페이지 시작
1  / 내용 = 한국 화랑협회가 주최해 여는 한국국제
2  / 내용 = "이달 23일, 일반에 온라인 뷰잉룸
3  / 내용 = 정동극장 '김주원의 사군자-생의 계절
4  / 내용 = 온라인 뷰잉룸으로 대체

지난해 열린
6  / 내용 = [스포츠서울 신재유기자] 경기도 포천
7  / 내용 = ◇뮤지컬 '캣츠'가 40주년 내한공연
9  / 내용 = 다사리문화기획학교 강의 야학[경기문화
10  / 내용 = 오프라인 전시 전격 취소
온라인 뷰잉
11  / 내용 = '가습기살균제 참사의 진실-빼앗긴 숨
12  / 내용 = 나점수 작가. 제공|아트스페이스3


13  / 내용 = 정부 방역 강화 지침에 따라 안전 우
14  / 내용 = '캣츠 마카롱 세트' 9월 한정 판매
15  / 내용 = 극단 신세계 18일 개막 '생활풍경'
16  / 내용 = [CBS노컷뉴스 문수경 기자]
(사진
17  / 내용 = 매·난·국·죽 '사군자' 소재로 시공
18  / 내용 = 서울시립미술관·권진규기념사업회 "상설
19  / 내용 = 극단 떼아뜨르 봄날이 재창작한 소포클
20  / 내용 = 부산비엔날레 온라인 개막
‘열장의 이
3번째 페이지 완료
**************************************************
4번째 페이지 시작
1  / 내용 = 김도희 개인전 ‘씨가 말랐대’
김도희
2  / 내용 = 리안갤러리 서울·리만머핀 동시에
20
3  / 내용 = 네이버TV '라이브 감상' 후원 리워
4  / 내용 = [파이낸셜뉴스] 신종 코로나 바이러스
5  / 내용 = 조정연 작가 '지고맺다'展 개최
[서
6  / 내용 = [정동극장 제공. 재판매 및 DB 금
8  / 내용 = (보은=연합뉴스) 전창해 기자 = 충
9  / 내용 = "자의적 저작권료 산정…공동대응 멈추
10  / 내용 = 문화체육관광부·한국문화관광연구원 추산
11  /

5  / 내용 = 극단 떼아뜨르 작품
인간 감정의 밑바
6  / 내용 = 승률 100%ㆍ강력한 우승후보에서 ‘
7  / 내용 = (서울=연합뉴스) 첫 산문집 '상관없
8  / 내용 = 재외공관 및 한국문화원 통해 서비스

9  / 내용 = 바이올리니스트 정경화와 첫 듀오 무대
10  / 내용 = (서울=연합뉴스) 첫 산문집 '상관없
11  / 내용 = 광주 시민 10인· 편의대원 7인 담
12  / 내용 = [서울=뉴시스]'일일유람 : 공예의 
13  / 내용 = 강헌 경기문화재단 대표.


[헤럴드
14  / 내용 = 9월 12일(토)~27일(일) 대명공
15  / 내용 = 총 700회 공연·전시회 개최…지역 
16  / 내용 = 2013년 등록문화재 제534호 지정
17  / 내용 = 12일(토), 13일(일) 서구문화회
18  / 내용 = 첫 산문집…"글쓰기와 음악, 위로한다
19  / 내용 = [서울=뉴시스]피아니스트 김선욱(사진
20  / 내용 = 국립극단은 '온라인 유료 극장' 추진
3번째 페이지 완료
**************************************************
4번째 페이지 시작
1  / 내용 = '코로나 일상 속 비대면 예술 지원 
2  / 내용 = 문체부, 사회관계장관회의서 '코로나 
3  / 내용 = KBS향과 지휘 데뷔 무대…브람스 교
4  / 내용 = 선화기독교미술관, 목원대에 작품 기증
5  / 내용 = 16일부터 22일까지 서울 갤러리M

6  / 내용 = SM 1호 클래식 연주자이자 SM 클
7  / 내용 = 9일부터 40주년 기념 내한공연
‘거
8  / 내용 = [서울=뉴시스] 뮤콘 온라인 2020
9  / 내용 = 장잉난, The Glory of th
10  / 내용 = 2019년 밀양대추축제
[밀양시 제공
11  / 내용 = (광명=연합뉴스) 김광호 기자 = 경
12  / 내용 = [헤럴드경제 = 서병기 선임기자]JT
13  / 내용 = 고사 위기 공연계…‘위드 코로나’ 시
14  / 내용 = 코리

20  / 내용 = ▶여성의 눈에 비친 개발자의 세계 『
2번째 페이지 완료
**************************************************
3번째 페이지 시작
1  / 내용 = 공포다. 몸이 조금만 이상해도 코로나
2  / 내용 = 한국인에겐 알려져 있지 않고, 일본에
3  / 내용 = 3년간 최대 1억2000만원 지원
[
4  / 내용 = 뮤지컬 '듀엣' 내달 23일 개막


6  / 내용 = 19일 오후 8시 전국 26개 극장

7  / 내용 = 3.1운동100주년 KBS 윤동주콘서
8  / 내용 = 국립극단 70년 역사 상 첫 온라인 
9  / 내용 = 창무국제공연예술제 개막작 'Here'
11  / 내용 = '캣츠'
[에스앤코 제공. 재판매 및
13  / 내용 = 김제 지평선축제
[연합뉴스 자료사진]
14  / 내용 = (서울=연합뉴스) 한미희 기자 = 여
15  / 내용 = 산자부, 문체부, 중기부 모두 전시·
16  / 내용 = (서울=연합뉴스) 대구대학교 중앙박물
17  / 내용 = 23일부터 5주간 매주 수요일
네이버
18  / 내용 = [서울시 제공. 재판매 및 DB 금지
19  / 내용 = (서울=연합뉴스) 대구대학교 중앙박물
20  / 내용 = 드랙퀸, 뮤지컬 트렌드로 재부상
'제
3번째 페이지 완료
**************************************************
4번째 페이지 시작
1  / 내용 = (서울=연합뉴스) 강종훈 기자 = ▲
2  / 내용 = 세계 접이식 플레이트카메라 특별전과 
4  / 내용 = 발레리노 안주원© 뉴스1

(서울=뉴
5  / 내용 = 지난 1년, '물의 도시' 베니스는 
6  / 내용 = 9월19일 전국 26곳서 상영…코로나
7  / 내용 = 고희안 신현필 [더브릿지 제공]



8  / 내용 = 1939년 창단한 세계적 발레단
한국
9  / 내용 = '선·열반의 노래'展, 서울 경인미술
10  / 내용 = 오프라인 공연의 감동 온라인으로…국내
11  / 

## 3. 시설 데이터 위/경도 누락 채우기

In [2]:
from bs4 import BeautifulSoup

In [3]:
browser = Chrome()

In [4]:
url = 'http://www.geocoding.co.kr/'

In [35]:
df = pd.read_csv('숙박위경도.csv', encoding = 'cp949')

In [43]:
df1 = df.loc[df.소재지도로명주소 == '없음']
df2 = df.loc[df.소재지도로명주소 != '없음']

In [69]:
df_null = list(df2.소재지지번주소)

In [70]:
xy_list = []

In [71]:
for row in trange(len(df_null)) :
    try :
        browser.get(url)
        element = browser.find_element_by_css_selector("#addr")
        element.send_keys(df_null[row])
        element.send_keys(Keys.RETURN)
        time.sleep(.7)
        위경도 = browser.find_element_by_css_selector("#td_lng_lat").text
        if len(위경도) == 0 :
            xy_list.append('재크롤링 필요')
            #print(row, '재크롤링 필요')
        else :
            xy = xy_list.append(위경도)
            #print(row, '완료', 위경도)    
        time.sleep(.5)
    except :
        continue

100%|████████████████████████████████████████████████████████████████████████████| 6420/6420 [2:42:24<00:00,  1.52s/it]


In [76]:
xy = pd.DataFrame(xy_list, columns = ['위경도'])

In [79]:
xy['위도'] = xy.위경도.str.split('\n').str[0].str[5:]

In [80]:
xy['경도'] = xy.위경도.str.split('\n').str[1].str[5:]

## 4. 역지오코딩

In [2]:
browser = Chrome()

In [3]:
df = pd.read_csv("가로수길key.csv")

In [4]:
df['위경도'] = df.가로수길시작위도.astype('str') + ' ' + df.가로수길시작경도.astype('str')

In [6]:
key_list = []
for row in range(df.shape[0]) :

    browser.get('https://map.kakao.com/')
    time.sleep(.5)
    element = browser.find_element_by_css_selector("#search\.keyword\.query")
    element.send_keys(df.위경도[row])
    element.send_keys(Keys.RETURN)
    time.sleep(.5)
    key = browser.find_element_by_css_selector("#localInfo\.map\.county").text
    print(key)
    key_list.append(key)

  0%|▎                                                                                 | 1/240 [00:02<09:23,  2.36s/it]

창원시 성산구


  1%|▋                                                                                 | 2/240 [00:04<09:21,  2.36s/it]

창원시 성산구


  1%|█                                                                                 | 3/240 [00:07<09:32,  2.41s/it]

...

수원시 팔달구


100%|███████████████████████████████████████████████████████████████████████████████▋| 239/240 [20:24<00:04,  4.91s/it]

수원시 장안구


100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [20:29<00:00,  5.12s/it]

수원시 영통구


In [8]:
df.key = key_list

In [16]:
df.at[df.loc[df.key == '마포구'].index, 'key'] = '서울특별시 마포구'
df.at[df.loc[df.key == '송파구'].index, 'key'] = '서울특별시 송파구'
df.at[df.loc[df.key == '유성구'].index, 'key'] = '대전광역시 유성구'
df.at[df.loc[df.key == '진주시'].index, 'key'] = '경상남도 진주시'
df.at[df.loc[df.key == '순창군'].index, 'key'] = '전라북도 순창군'
df.at[df.loc[df.key == ''].index, 'key'] = '창원시 마산합포구'